# 데이터 불러오기 등

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

In [ ]:
ratings = pd.read_csv('/data/kakao_data/ratings_two.csv')
ratings = ratings[['user_id', 'cafe', 'rating']]
ratings['rating'] = ratings['rating'].astype(int)           # timestamp 제거

In [ ]:
user_cafe_rating = ratings.pivot(index='user_id', columns='cafe', values='rating')
print(user_cafe_rating.shape)
cafe_user_rating = ratings.pivot(index='cafe', columns='user_id', values='rating')
print(cafe_user_rating.shape)

(6842, 5165)
(5165, 6842)


# surprise를 활용한 train/test

https://hanshuginn.blogspot.com/2021/04/svd-python-svdsingular-value.html

In [ ]:
!pip install surprise
from surprise import SVD
from surprise import Dataset
from surprise import dump
from surprise import accuracy
from surprise import Reader
import pandas as pd 
from collections import defaultdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633976 sha256=eab171104172957a7397572280c4639310dc5cbd8a4949399ba8f133f79edff7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## 전처리

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32622 entries, 0 to 32621
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          32622 non-null  object 
 1   user_name        32539 non-null  object 
 2   user_review_num  32622 non-null  int64  
 3   user_rating_avg  32622 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1019.6+ KB


In [ ]:
ratings.groupby('user_id',as_index=False).mean().rename(columns = {'rating':'user_rating_avg'})

,user_id,user_rating_avg
0,100bln7,5.000000
1,100ck0j,2.333333
2,100i22n,3.333333
3,100ooji,5.000000
4,100q04c,3.736842
...,...,...
6837,vv2c9h,4.000000
6838,vv73qf,2.000000
6839,vvcfbq,3.500000
6840,vvkrvp,5.000000


In [ ]:
ratings_mean = ratings.merge(ratings.groupby('user_id',as_index=False).mean().rename(columns = {'rating':'user_rating_avg'}), on='user_id', how='left')
print(len(ratings_mean))

23308


In [ ]:
print(len(ratings))

23308


In [ ]:
ratings_mean.head()

,user_id,cafe,rating,user_rating_avg
0,100bln7,3F로비 한강로3가,5,5.000000
1,100bln7,카페로비 한강로3가,5,5.000000
2,100ck0j,공차 이대익스프레스점 대현동,1,2.333333
3,100ck0j,투썸플레이스 숙대입구역점 남영동,1,2.333333
4,100ck0j,폴바셋 방배역점 방배동,5,2.333333


In [ ]:
ratings_mean['rating_adjusted'] = ratings_mean['rating'] - ratings_mean['user_rating_avg']

## SVD 모델

In [ ]:
### 학습데이터 포멧팅 from DataFrame
DfOrgData = ratings_mean[['user_id','cafe','rating_adjusted']]

r_min = DfOrgData['rating_adjusted'].min()
r_max = DfOrgData['rating_adjusted'].max()
reader = Reader(rating_scale=(r_min, r_max))
data = Dataset.load_from_df(DfOrgData[['user_id', 'cafe', 'rating_adjusted']],reader)

trainset = data.build_full_trainset()
testset = trainset.build_testset()

algo = SVD()

### trainset으로 SVD 학습
algo.fit(trainset)

In [ ]:
### testset으로 RMSE 측정
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7240


0.7240235389978461

### 최적의 parameter 찾기

In [ ]:
from surprise.model_selection import GridSearchCV, cross_validate

param_grid = {'n_factors': [10, 25, 50, 75]} 
gs = GridSearchCV(algo_class=SVD, measures=['RMSE','MAE'], param_grid=param_grid) 
gs.fit(data) 
print('\n###################') 
print('Best Score (RMSE) :', gs.best_score['rmse']) 
print('Best Score (MAE) :', gs.best_score['mae']) 
print('Best Parameters :', gs.best_params['rmse']) 
print('#####################')


###################
Best Score (RMSE) : 1.0251174466972965
Best Score (MAE) : 0.737173817030226
Best Parameters : {'n_factors': 25}
#####################


# Predict

## user별 평균 조정  
user별로 평균을 뺀 뒤, 결측값을 0으로 채우기

In [ ]:
matrix = user_cafe_rating.to_numpy()

In [ ]:
user_ratings_mean1 = np.nanmean(matrix, axis=1)

In [ ]:
len(user_ratings_mean1)

6842

In [ ]:
user_ratings_mean1

array([5.        , 2.33333333, 3.33333333, ..., 3.5       , 5.        ,
       3.        ])

In [ ]:
matrix_user_mean = matrix - user_ratings_mean1.reshape(-1,1)

In [ ]:
matrix_user_mean

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [ ]:
matrix_user_mean = pd.DataFrame(matrix_user_mean).fillna(0).to_numpy()
matrix_user_mean.shape

(6842, 5165)

## SVD

In [ ]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 25)

In [ ]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(6842, 25)
(25,)
(25, 5165)


In [ ]:
sigma = np.diag(sigma)

In [ ]:
sigma.shape

(25, 25)

In [ ]:
user_ratings_mean1.reshape(-1,1)

array([[5.        ],
       [2.33333333],
       [3.33333333],
       ...,
       [3.5       ],
       [5.        ],
       [3.        ]])

## predict

In [ ]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean1.reshape(-1, 1)

In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, 
                            columns = user_cafe_rating.columns,
                            index = user_cafe_rating.index)
df_svd_preds.head()

cafe,0125커피바 홍은동,044워리어스 당산동1가,089커피앤베이커리 화곡동,1028커피엔디저트 성내동,10인치샌드위치&커피 봉천동,125coffee 갈현동,139COFFEE 전농동,1980벽돌집 신정동,1월의윤슬 내발산동,205도씨 명륜2가,...,히어로보드게임카페 홍대2호점 동교동,히어로스터 신도림동,히어커피 양재동,히얼스유알커피 삼성동,히자우 홍제동,히포커피 대학동점 신림동,히포커피서울대점 서울대점 신림동,히히냥냥 역삼동,힐브레드 마곡동,힘들땐마카롱 신당동
user_id,,,,,,,,,,,,,,,,,,,,,
100bln7,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
100ck0j,2.333333,2.333387,2.333312,2.333306,2.333487,2.333333,2.333333,2.333333,2.333218,2.33443,...,2.333551,2.333341,2.333332,2.333333,2.333336,2.333367,2.332367,2.333398,2.333110,2.333020
100i22n,3.333333,3.332647,3.333378,3.334760,3.333486,3.333334,3.333333,3.333337,3.334057,3.33152,...,3.335477,3.337723,3.333341,3.333333,3.333334,3.333350,3.348829,3.334615,3.334088,3.332764
100ooji,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
100q04c,3.736842,3.735949,3.736897,3.745772,3.737025,3.736843,3.736842,3.736851,3.735759,3.73400,...,3.716632,3.732389,3.736884,3.736842,3.736562,3.736645,3.734394,3.737515,3.739241,3.727903


In [ ]:
df_svd_preds.to_csv('/content/drive/MyDrive/modeling/model/승주MF/df_svd_preds_two.csv')

# Recommender

In [ ]:
def recommend_cafe(df_svd_preds, user_id, ori_ratings_df, num_recommendations=5):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = df_svd_preds.loc[user_id,:].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id][['cafe','rating']]
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출

    recommendations = ori_ratings_df[~ori_ratings_df['cafe'].isin(user_data)]['cafe']

    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(recommendations).merge(pd.DataFrame(sorted_user_predictions),right_index = True,left_index=True,how='right')
    recommendations.drop('cafe',axis=1,inplace=True)

    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

In [ ]:
def recommend_cafe_address(df_svd_preds, user_id, ori_ratings_df, num_recommendations=5, address='영등포구'):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = df_svd_preds.loc[user_id,:].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id][['cafe','rating']]
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출

    recommendations = ori_ratings_df[~ori_ratings_df['cafe'].isin(user_data)]['cafe']


    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(recommendations).merge(pd.DataFrame(sorted_user_predictions),right_index = True,left_index=True,how='right')

    # 이미 갔던 곳 제외하기 위한 코드 추가
    user_data.reset_index(inplace=True, drop=True)
    for i in range(len(user_data)):
      recommendations = recommendations[recommendations.index!=user_data.loc[i,'cafe']]
    
    recommendations = pd.merge(recommendations, public_df.set_index('cafe')[['시군구명','도로명주소']], left_index=True,right_index=True,how='left')
    recommendations.drop('cafe',axis=1,inplace=True)

    ## address 부분
    recommendations = recommendations[recommendations['시군구명']==address]

    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

In [ ]:
already_rated, predictions = recommend_cafe(df_svd_preds,user_id='10l9o2l', ori_ratings_df=ratings,num_recommendations=10)

In [ ]:
predictions

,Predictions,시군구명,도로명주소
커피브론즈 안국동,3.604750,종로구,서울특별시 종로구 율곡로5길 20
블루보틀커피코리아삼청 소격동,3.582769,종로구,서울특별시 종로구 삼청로2길 40-3
밀랑스 대흥동,3.581247,마포구,서울특별시 마포구 대흥로28길 5
스타벅스 노량진동점 노량진동,3.570263,동작구,서울특별시 동작구 노량진로 190
정이정 종암동,3.566087,성북구,서울특별시 성북구 종암로 82-15
튠업카페 이문동,3.564785,동대문구,서울특별시 동대문구 이문로9길 46
"그리고스튜디오,흑석커피 흑석동",3.564728,동작구,서울특별시 동작구 서달로14가길 5
오프커피 성수동2가,3.564477,성동구,서울특별시 성동구 연무장길 29-17
언더프레셔 삼성동,3.563717,강남구,서울특별시 강남구 영동대로 513
디벙크 합정동,3.563325,마포구,서울특별시 마포구 성지1길 30


In [ ]:
already_rated, predictions = recommend_cafe_address(df_sgd_preds, user_id='10l9o2l', ori_ratings_df=ratings,num_recommendations=10,
                                                    address='서대문구')

In [ ]:
predictions

,Predictions,시군구명,도로명주소
투썸플레이스 신촌연세로점 창천동,3.523958,서대문구,서울특별시 서대문구 연세로 28
투썸플레이스신촌점 신촌점 창천동,3.523146,서대문구,서울특별시 서대문구 연세로 1
증가로커피공방 남가좌동,3.519637,서대문구,서울특별시 서대문구 증가로10길 36-55
ATOZCAFE 창천동,3.516776,서대문구,서울특별시 서대문구 연세로4길 30
에스페란자로스터즈 홍은동,3.515637,서대문구,서울특별시 서대문구 증가로4길 58-15
할리스커피 신촌연세로점 창천동,3.515580,서대문구,서울특별시 서대문구 연세로 5
설빙신촌점 신촌점 창천동,3.508160,서대문구,서울특별시 서대문구 연세로 19
빽다방 신촌역점 창천동,3.506580,서대문구,서울특별시 서대문구 연세로2길 6
논탄토 신촌점 창천동,3.505155,서대문구,서울특별시 서대문구 신촌로 87-4
쥬씨 신촌점 창천동,3.504792,서대문구,서울특별시 서대문구 연세로 30
